# Backtest HKV

In [ ]:
import pandas as pd 
from datetime import datetime
import openstef
from openstef.data_classes.model_specifications import ModelSpecificationDataClass
from openstef.data_classes.prediction_job import PredictionJobDataClass 
from openstef.pipeline.train_create_forecast_backtest import train_model_and_forecast_back_test
from openstef.feature_engineering.weather_features import calculate_dni, calculate_gti

from tqdm.autonotebook import trange

# Set plotly as the default pandas plotting backend
pd.options.plotting.backend = 'plotly'

from get_rcdata import get_rcdataframe, find_nearest

import xarray as xr

In [ ]:
data = xr.open_dataset('data/raycast_test_knmi.nc')
knmi_stations = pd.read_csv('data/knmi_stations.csv', index_col=0)

In [ ]:
lat = 53.445448
lon = 5.7226894
station = find_nearest(lat, lon, knmi_stations)
lead = 3

In [ ]:
raycastdf = get_rcdataframe(station, data, lead=lead).drop(columns=['lead_time'])
display(raycastdf)

In [ ]:
raycastdf.index = raycastdf.index + pd.DateOffset(minutes=15*lead)

raycastdf.columns = [f"raycast_{lead}h_{col}" for col in raycastdf.columns.get_level_values(1)]
raycastdf.index = pd.to_datetime(raycastdf.index, utc=True)

display(raycastdf)


## Define the prediction job and load the data


In [ ]:
# Define properties of training/prediction.

# Quantiles
quantiles = [0.05, 0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90, 0.95]

# Sun-heavy         lat,lon: 53.445448, 5.7226894
# Wind-heavy        lat,lon: 52.515094, 5.4768915
# Sun-Wind-heavy    lat,lon: 53.325670, 5.7532568
# Consumption-heavy lat,lon: 52.30096,  5.04536
pj = {
    "lat":53.445448,
    "lon":5.7226894,
    "id": 307,
    "name": "Back_test_prediction_job",
    "typ": "demand",
    "model": "xgb",
    "horizon_minutes": 2880,  # How many minutes in the future should be forecasted
    "resolution_minutes": 15,  # In what timestep should be forecasted
    "train_components": 1,
    "sid": "Back_test",
    "created": datetime.now(),
    "description": "",
    "forecast_type": "demand",
    "quantiles": quantiles,
    "hyper_params": {},
    "feature_names": None, 
    "model_type_group": "default",
}

In [ ]:
# Data files:

# 2020_data_sun_heavy.csv
# 2020_data_wind_heavy.csv
# 2020_data_wind_sun_heavy.csv
# 2020_data_consumption_heavy.csv

# 2023_data_sun_heavy.csv
# 2023_data_wind_heavy.csv
# 2023_data_wind_sun_heavy.csv
# 2023_data_consumption_heavy.csv

input_data=pd.read_csv("data/2020_data_sun_heavy.csv", index_col=0, parse_dates=True)
display(input_data)
print(input_data.columns)

In [ ]:
for col in raycastdf.columns:
    input_data[col] = input_data.index.map(raycastdf[col].to_dict())

In [ ]:
input_data.dropna()

In [ ]:
calculate_dni(input_data["raycast_3h_0.5"], pj)
input_data["dni_raycast_3h_0.5"] = calculate_dni(input_data["raycast_3h_0.5"], pj)
input_data["gti_raycast_3h_0.5"] = calculate_gti(input_data["raycast_3h_0.5"], pj)

## Perform the backtest

In [ ]:
backtest_iterations = 10
backtest_horizon = 24
backtest_folds = 4

forecasts: list[pd.DataFrame] = []

for _ in trange(backtest_iterations):
    backtest_result = train_model_and_forecast_back_test(
        PredictionJobDataClass(**pj),
        ModelSpecificationDataClass(**pj),
        input_data,
        training_horizons=[backtest_horizon],
        n_folds=backtest_folds,
    )
    forecast = backtest_result[0]
    forecast = forecast.loc[forecast["horizon"] == backtest_horizon]
    forecast = forecast.drop(
        columns=[
            "pid",
            "customer",
            "description",
            "type",
            "algtype",
            "tAhead",
            "horizon",
        ]
    )
    forecasts.append(forecast)

In [ ]:
forecasts_combined = pd.concat(forecasts, axis="columns")
forecast_median = pd.DataFrame()
for column in forecasts[0].columns:
    forecast_median[column] = forecasts_combined[column].median(axis="columns")

## Evaluate the results 


In [ ]:
import plotly.graph_objs as go
import numpy as np
def plot_percentiles(timeseries: pd.DataFrame):
    # Generate traces of Percentiles, fill below
    figure = go.Figure()
    for i, percentile in enumerate(np.sort([x for x in timeseries.columns if x[0] == "q"])):
        fill = None if i == 0 else "tonexty"
        figure.add_trace(
            go.Scatter(
                x=timeseries.index,
                y=timeseries[percentile],
                fill=fill,
                name=percentile,
                line=dict(width=1),
            )
        )

    # Add historic load
    figure.add_trace(
        go.Scatter(
            x=timeseries.index,
            y=timeseries["realised"],
            name="realised",
            line=dict(color="red", width=2),
        )
    )

    figure.update_layout(title="Backtest - Prognoses vs Realisatie")

    return figure

In [ ]:
plot_percentiles(forecast_median)

In [ ]:
openstef.metrics.figure.plot_feature_importance(backtest_result[1][0].feature_importance_dataframe)

In [ ]:
display((backtest_result[1][0].feature_importance_dataframe.loc["gti_raycast_3h_0.5"]))

In [ ]:
input_data2 = input_data.copy(deep=True)
input_data2["radiation"] = input_data2["radiation"]/max(input_data2["radiation"])
input_data2["raycast_3h_0.5"] = input_data2["raycast_3h_0.5"]/max(input_data2["raycast_3h_0.5"].dropna())
input_data2[["radiation", "raycast_3h_0.5"]].plot()